In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# root_path = '/content/drive/My Drive/Google Colab/'

In [0]:
import os
os.chdir('/content/drive/My Drive/Google Colab/')

In [0]:
import os
import numpy as np
import datetime
import time
import pandas
import pdb
from scipy.interpolate import interp1d
import pickle
import statistics
from scipy.signal import butter, filtfilt
from scipy.stats import zscore, kurtosis, skew
from utils import calculate_ellipse_area, create_ellipse, energy_peak, zero_crossing_rate, energy, sma, percentile

In [0]:
# file = "TJR/ID_303/36M_WALK_TEST/accelerometer.csv"
file = "TJR/App_Data.csv"
# walkBAC = pandas.read_csv(file, skiprows = 1)
walkBAC = pandas.read_csv(file)

In [0]:
sensorData = walkBAC.dropna()

In [0]:
# sensorData

In [0]:
finalData = sensorData.drop_duplicates()

In [0]:
date_time = finalData["TimeStamp"].str.split(" ", n = 1, expand = True)

In [0]:
# date_time

In [0]:
finalData["Time"]= date_time[1] 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
finalData.drop(columns =["TimeStamp"], inplace = True)
# inplace does changes in the original dataset

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
# finalData

In [0]:
# sensorData = walkBAC[['X', 'Y', 'Z']]

In [0]:
# sensorData

In [0]:
# semgementation_seconds = 8
# signaltype = '36M_WALK_TEST'

In [0]:
def butter_lowpass_filter(data):
  T = 5.0         # Sample Period
  fs = 50.0       # sample rate, Hz
  cutoff = 2      # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
  nyq = 0.5 * fs  # Nyquist Frequency
  order = 3       # sin wave can be approx represented as quadratic
  n = int(T * fs) # total number of samples

  noise = 1.5*np.cos(9*2*np.pi*T) + 0.5*np.sin(12.0*2*np.pi*T)
  normal_cutoff = cutoff / nyq

  x = data['X'] + noise
  y = data['Y'] + noise
  z = data['Z'] + noise

  b, a = butter(order, normal_cutoff, btype='low', analog=False)
  data['X'] = filtfilt(b, a, x)
  data['Y'] = filtfilt(b, a, y)
  data['Z'] = filtfilt(b, a, z) 
  return data

In [0]:
data = butter_lowpass_filter(finalData)
print(data.head())

     ID         X          Y         Z Condition          Time
0   303 -0.453663  11.347826  2.110922   Healthy  14:35:36.980
4   303 -0.495761  11.270878  2.056700   Healthy  14:35:36.982
7   303 -0.539317  11.187680  1.998793   Healthy  14:35:36.984
11  303 -0.582127  11.104188  1.940741   Healthy  14:35:36.985
14  303 -0.622249  11.025850  1.885853   Healthy  14:35:36.987


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [0]:
# X_train = data[data['ID'] < 309]
# y_train = X_train['Condition']
# X_train = X_train.drop(['Condition','TimeStamp'],axis=1)

# X_test = data[data['ID'] >= 309]
# y_test = X_test['Condition']
# X_test = X_test.drop(['Condition','TimeStamp'],axis=1)

In [0]:
# data.iloc[0]['Time']

In [0]:
# def segmenting(data):
#     start = 0
#     end = 0
#     i = 0
#     for s in data['Time']:
#         if data.iloc[i]['ID'] == data.iloc[i+1]['ID']:
# #             print(s)
# #             print(data.iloc[start]['Time'])
# #             diff = s - data.iloc[start]['Time']
#             r = data.iloc[start]['Time']
# #             print(r)
#             t1 = pandas.to_datetime(s)
# #             print(s)
#             t2 = pandas.to_datetime(r)
# #             print(r)
#             diff = pandas.Timedelta(t1 - t2).total_seconds()
# # #             diff = (s - data.iloc[start]['Time']).astype('timedelta64[h]')
#             print(diff)
#             i += 1
# #             if(diff == seg_seconds)
# #         print(i, s)

In [0]:
# segmenting(data)

In [0]:
def find_features(uid, x, y, z, c, t):
    if c == 'Healthy':
        problem = 0
    elif c == 'OA':
        problem = 1
    else:
        problem = 2

    # Setting values of x, y and z
    x_seg = x
    y_seg = y
    z_seg = z

    # For average
    x_avg = np.average(x_seg)
    y_avg = np.average(y_seg)
    z_avg = np.average(z_seg)

    # For range
    x_range = max(x_seg) - min(x_seg)
    y_range = max(y_seg) - min(y_seg)
    z_range = max(z_seg) - min(z_seg)

    # For standard deviation
    x_std = statistics.stdev(x_seg)
    y_std = statistics.stdev(y_seg)
    z_std = statistics.stdev(z_seg)

    # For RMS
    x_rms = np.sqrt(np.mean(x_seg ** 2))
    y_rms = np.sqrt(np.mean(y_seg ** 2))
    z_rms = np.sqrt(np.mean(z_seg ** 2))      

    # For skew     
    x_sk = skew(x_seg)
    y_sk = skew(y_seg)
    z_sk = skew(z_seg)

    # For energy
    x_energy = energy(x_seg)
    y_energy = energy(y_seg)
    z_energy = energy(z_seg)

    # For kurtosis
    x_kurt = kurtosis(x_seg)
    y_kurt = kurtosis(y_seg)
    z_kurt = kurtosis(z_seg)

    # # For energy and frequency peak
    # x_enpeak, x_freqpeak = energy_peak(x_seg)
    # y_enpeak, y_freqpeak = energy_peak(y_seg)
    # z_enpeak, z_freqpeak = energy_peak(z_seg)      

    # # For signal magnitude area
    # sig_area = sma(x_seg,y_seg,z_seg)

    Walk = []
    Walk.append([uid,x_avg,y_avg,z_avg,x_range,y_range,z_range,x_std,y_std,z_std,x_rms,y_rms,z_rms,\
                 x_sk,y_sk,z_sk,x_energy,y_energy,z_energy,\
                #  x_kurt,y_kurt,z_kurt,
                #  x_enpeak,y_enpeak,z_enpeak,\
                #  x_freqpeak,y_freqpeak,z_freqpeak,sig_area,\
                 problem])
                 
    WholeWalk.append(Walk)

In [0]:
data.count()

ID           45148
X            45148
Y            45148
Z            45148
Condition    45148
Time         45148
dtype: int64

In [0]:
WholeWalk = []
def segmenting(data):
    start = 0
    end = 0
    i = 0
    for s in data['Time']:
        if i < 45147:
            if data.iloc[i]['ID'] == data.iloc[i+1]['ID']:
    #             print(s)
    #             print(data.iloc[start]['Time'])
    #             diff = s - data.iloc[start]['Time']
                r = data.iloc[start]['Time']
    #             print(r)
                t1 = pandas.to_datetime(s)
    #             print(s)
                t2 = pandas.to_datetime(r) 
    #             t3 = pandas.to_timedelta(r) + pandas.Timedelta(seconds=8)
    #             print(t3)
                diff = pandas.Timedelta(t1 - t2).total_seconds()
    #             diff = pandas.Timedelta(t1 - t2)
#                 print(diff)
    #             print(diff - seg_seconds)
    #             diff = t1 - t2
    #             print(pandas.to_timedelta(diff, unit='s'))
    #             if pandas.to_timedelta(diff, unit='s') < pandas.to_timedelta(t3, unit='s'):
    #             if t3 - diff > 0
                if diff - seg_seconds >= 0:
                    end = i
    #                 print(end)
                    uid = data.iloc[i]['ID']
                    x = data["X"][start:end]
                    y = data["Y"][start:end]
                    z = data["Z"][start:end]
                    c = data.iloc[i]["Condition"]
                    find_features(uid, x, y, z, c)
                    start = end + 1
                    # start = start + int((end - start)*3/4 + 1)   # 0.25 overlapping
                    # start = start + int((end - start)*2/4  + 1)     # 0.5 overlapping
                    # start = start + int((end - start)*1/4  + 1)     # 0.75 overlapping
                    # start = int(end/4 + 1)     # 0.75 overlapping previously (wrong)
                    #ignores the segment which is less than 8 seconds
    # #             diff = (s - data.iloc[start]['Time']).astype('timedelta64[h]')
    #             print(diff)
                i += 1
            else:
                i += 1
        else:
            break
#     return segments
#             if(diff == seg_seconds)
#         print(i, s)

In [0]:
# segmenting(data)

In [0]:
# from scipy import stats


# def segment_signal(data,window_size = 10):
#     segments = np.empty((0,window_size,3))
#     labels = np.empty((0))
#     for (start, end) in windows(data["TimeStamp"], window_size):
        
#         if(len(data["TimeStamp"][start:end]) == window_size):
#             segments = np.vstack([segments,np.dstack([x,y,z])])
#             labels = np.append(labels,stats.mode(data["Condition"][start:end])[0][0])
#     return segments, labels

In [0]:
# (df.fr-df.to).astype('timedelta64[h]')

In [0]:
# seg_seconds = 8.00
seg_seconds = 5.00
# seg_seconds = 10.00
# seg_seconds = 2.00
# samples = 128

# samplingType = 'segments'
# overlap_rate = 0.5

segmenting(data)

In [0]:
# WholeWalk

In [0]:
z = len(WholeWalk)

In [0]:
z

92

In [0]:
matrix = np.array(WholeWalk).reshape(z, 20) 

In [0]:
# matrix

In [0]:
model_data = pandas.DataFrame(matrix, columns=['ID','X_Avg','Y_Avg','Z_Avg','X_Range','Y_Range','Z_Range','X_SD','Y_SD','Z_SD','X_RMS','Y_RMS','Z_RMS','X_Skew','Y_Skew','Z_Skew','X_Energy','Y_Energy','Z_Energy','Class'])

In [0]:
X_train = model_data[model_data['ID'] < 309]
y_train = X_train['Class']
X_train = X_train.drop(['Class'],axis=1)

X_test = model_data[model_data['ID'] >= 309]
y_test = X_test['Class']
X_test = X_test.drop(['Class'],axis=1)

In [0]:
# y_train = X_train['Class']
# X_train = X_train.drop(['Class'],axis=1)

In [0]:
# WholeWalk

In [0]:
# WholeWalk = []
# segmenting(X_test)
# X_test = DataFrame(WholeWalk, columns=['X_Avg','Y_Avg','Z_Avg','X_Range','Y_Range','Z_Range','X_SD','Y_SD','Z_SD','X_RMS','Y_RMS','Z_RMS','X_Skew','Y_Skew','Z_Skew','X_Energy','Y_Energy','Z_Energy','Class'])

In [0]:
# y_test = X_train['Class']
# X_test = X_train.drop(['Class'],axis=1)

In [0]:
# WholeWalk

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix, accuracy_score

In [0]:
parameters = {'bootstrap': True,
              'min_samples_leaf': 3,
              'n_estimators': 50, 
              'min_samples_split': 10,
              'max_features': 'sqrt',
              'max_depth': 6,
              'max_leaf_nodes': None}

In [0]:
RF_model = RandomForestClassifier(**parameters)

In [0]:
RF_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=6, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
y_pred = RF_model.predict(X_test)
score = accuracy_score(y_test ,y_pred)
print(score)

0.5483870967741935


In [0]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      0.71      0.83         7
         1.0       0.00      0.00      0.00        12
         2.0       0.50      1.00      0.67        12

    accuracy                           0.55        31
   macro avg       0.50      0.57      0.50        31
weighted avg       0.42      0.55      0.45        31



In [0]:
from sklearn.svm import SVC

In [0]:
clf = SVC(gamma='auto')
clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
y_pred = clf.predict(X_test)
score = accuracy_score(y_test ,y_pred)
print(score)

0.3870967741935484


In [0]:
import xgboost as xgb

# xg = xgb.XGBClassifier(alpha=0.1, reg_lambda=0.01)
xg = xgb.XGBClassifier()
xg

XGBClassifier(alpha=0.1, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=0.01, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
# XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#        colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
#        max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
#        n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
#        reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
#        silent=True, subsample=1)

In [0]:
xg.fit(X_train, y_train)
y_pred = xg.predict(X_test)
accuracy_score(y_test, y_pred)

0.5483870967741935